## 1.爬取大学所属地区链接(国内大学)

In [32]:
import requests
from lxml import etree
import re
import json
import pandas as pd
url="https://daxue.911cha.com/"
headers={
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "cookie":"Hm_lvt_2e69b379c7dbfdda15f852ee2e7139dc=1686271732,1686818515; Hm_lpvt_2e69b379c7dbfdda15f852ee2e7139dc=1686819239; t=e90d7bb8d83fde443ef9aa2347096872; r=7492"
}
response=requests.get(url,headers=headers)

国内大学链接 =re.findall(r'<a href="\./s(.*?)">.*?</a>', response.text)
国内大学地区 =re.findall(r'<a href="\./s.*?">(.*?)</a>', response.text)

fo ={'地区链接':国内大学链接,'地区':国内大学地区} 
fd = pd.DataFrame(fo)
fd['地区链接']=fd['地区链接'].map(lambda x:'https://daxue.911cha.com/s'+x)
fd.to_csv('国内地区链接.csv',header = True,index=0)   

## 2.根据大学所属地区链接爬取大学详情页链接（1973条数据）

In [61]:
data1=pd.read_csv('国内地区链接.csv')
url1=data1['地区链接'].tolist()

In [63]:
国内大学名称=[]
国内大学详情页链接=[]
for url in url1:
    response=requests.get(url,headers=headers)
    大学名称 =re.findall(r'<li><a href="\./.*?">(.*?)</a></li>', response.text)
    大学详情页链接 =re.findall(r'<li><a href="\.(/.*?)">.*?</a></li>', response.text)
    国内大学名称+=大学名称
    国内大学详情页链接+=大学详情页链接

In [71]:
fo ={'大学名称':国内大学名称,'大学详情页链接':国内大学详情页链接} 
fd = pd.DataFrame(fo)
fd['大学详情页链接']=fd['大学详情页链接'].map(lambda x:'https://daxue.911cha.com'+x)
fd.to_csv('国内大学详情页链接.csv',header = True,index=0)   

In [73]:
fd

,大学名称,大学详情页链接
0,北京大学,https://daxue.911cha.com/1.html
1,清华大学,https://daxue.911cha.com/2.html
2,中国人民大学,https://daxue.911cha.com/3.html
3,北京师范大学,https://daxue.911cha.com/24.html
4,北京航空航天大学,https://daxue.911cha.com/6.html
...,...,...
1968,香港大学,https://daxue.911cha.com/1945.html
1969,香港中文大学,https://daxue.911cha.com/1946.html
1970,香港理工大学,https://daxue.911cha.com/1947.html
1971,香港科技大学,https://daxue.911cha.com/1948.html


国内共有1973所大学

## 3.根据链接爬取大学详情页信息

In [1]:
import requests
from lxml import etree
import re
import json
import pandas as pd
import time
data=pd.read_csv('国内大学详情页链接.csv')
data['大学详情页链接']=data['大学详情页链接'].map(lambda x:x.replace('" class="inlink',""))
urls=data['大学详情页链接'].tolist()

In [26]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

option=Options()
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument("--log-level=3")
option.add_argument("--use-fake-ui-for-media-stream")
driver = Chrome(options=option)

f = open('国内大学.csv', 'a', encoding='utf8')
f.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format("院校名称",'院校类型','是否是211', '是否是985', 
                                                                '是否有研究生院','院校隶属','所在地区',
                                                                '学校网址','特殊招生','学校排名',
                                                               '科学研究','星级','总分','人才培养',
                                                                '综合声誉','办学层次'))
for i in range(len(urls):
    try:
        driver.get(urls[i])
        time.sleep(2)
        text = driver.page_source
        院校名称=data['大学名称'].tolist()[i]
        院校类型 ="".join(re.findall(r'院校类型：</span>\s*(.*?)\s*<', text))
        是否是211="".join(re.findall(r'title="211工程">\s*(\d+)\s*<', text))
        是否是985="".join(re.findall(r'title="985工程">\s*(\d+)\s*<', text))
        是否有研究生院="".join(re.findall(r'title="有研究生院">\s*(.*?)\s*<',text))
        院校隶属="".join(re.findall(r'院校隶属：</span>(.*?)<',  text))
        
        所在地区="".join(re.findall(r'所在地区：</span>(.*?)<', text))
        if 所在地区=="":
            所在地区="".join(re.findall(r'所在地区：</span><a href="\./.*?">(.*?)<', text))
            
        学校网址="".join(re.findall(r'学校网址：</span><a href="(.*?)"',  text))
        特殊招生=re.findall(r'特殊招生：</span>(.*?)<',text)
        学校排名="".join(re.findall(r'学校排名：</span>第(\d+)名',text))
        科学研究="".join(re.findall(r'科学研究：</span>(.*?)分',  text))
        星级="".join(re.findall(r'星级：</span><span class="red f18">(.*?)<',text)).count("★")
        总分="".join(re.findall(r'总分：</span>(.*?)分<br',  text))
        人才培养="".join(re.findall(r'人才培养：</span>(.*?)分<',  text))
        综合声誉="".join(re.findall(r'综合声誉：</span>(.*?)分<',  text))
        办学层次="".join(re.findall(r'办学层次：</span>(.*?)<',  text))
        f.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(院校名称,院校类型,是否是211, 是否是985, 
                                                                        是否有研究生院,院校隶属,所在地区,
                                                                        学校网址,特殊招生,学校排名,
                                                                       科学研究,星级,总分,人才培养,
                                                                        综合声誉,办学层次))
        driver.back()
        print(i)
    except:
        time.sleep(15)
        print(f'url为{i}的数据爬取出错')
        driver.back()
        continue
f.close()
print('已完成')

已完成
